In [106]:
import pandas as pd
import numpy as np
import os
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
# spark = SparkSession.builder \
#     .master("local[1]") \
#     .appName("SparkApp") \
#     .getOrCreate()

In [3]:
if os.getcwd().split("/")[-1] != "FeaturesExtracted":
    if os.getcwd().split("/")[-1] == "DataSources":
        os.chdir("../../SpotGenTrack/FeaturesExtracted")
    else:
        os.chdir("SpotGenTrack/FeaturesExtracted")

finalDF = pd.read_csv("final_df.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,205,209,217,229,245) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
selected_features_with_popularity = ["acousticness", "artist_popularity", "duration_ms", "energy", 
                     "entropy_energy", "followers", "loudness", "speechiness", "popularity"]

selected_features_with_popularity_alt = ["artist_popularity", "entropy_energy", "followers", 
                                         "loudness", "speechiness", "popularity"]

selected_features_no_popularity = ["acousticness", "artist_popularity", "duration_ms", "energy", 
                     "entropy_energy", "followers", "loudness", "speechiness"]

selected_features_no_popularity_alt = ["artist_popularity", "entropy_energy", "followers", 
                                         "loudness", "speechiness"]

splittingDF = finalDF[selected_features_with_popularity]
print(f"Before drop NaN: {splittingDF.shape}")
splittingDF.dropna(inplace=True)
print(f"After drop NaN: {splittingDF.shape}")
train, test = train_test_split(splittingDF)

Before drop NaN: (101939, 9)
After drop NaN: (74959, 9)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [47]:
X_train = train[selected_features_no_popularity][:20000]
X_test = test[selected_features_no_popularity][:4000]
y_train = train["popularity"][:20000]
y_test = test["popularity"][:4000]

In [133]:
def train_test_model(X_train, X_test, y_train, y_test, model="Linear Regression"):
    if model == "Linear Regression":
        clf = linear_model.LinearRegression()
        clf.fit(X_train, y_train)
    elif model == "Decision Tree Regression": 
        parameters = {'max_depth':[3, 5, 8, 10, 13, 15]}
        
        dtr = DecisionTreeRegressor()
        clf = GridSearchCV(dtr, parameters, scoring = "r2")
        clf.fit(X_train, y_train)
    elif model == "Random Forest Regression":
        parameters = {'max_depth':[3, 5, 8, 10, 13, 15]}
        
        rfr = RandomForestRegressor()
        clf = GridSearchCV(rfr, parameters, scoring = "r2")
        clf.fit(X_train, y_train)
        
    y_pred = clf.predict(X_test)
    
    print(f"MAE for {model}: {mean_absolute_error(y_test, y_pred)}")
    print(f"MSE for {model}: {mean_squared_error(y_test, y_pred)}")
    print(f"Rˆ2 for {model}: {r2_score(y_test, y_pred)}")
    
    return clf

In [134]:
train_test_model(X_train, X_test, y_train, y_test)

MAE for Linear Regression: 8.862932881888723
MSE for Linear Regression: 131.6172252663196
Rˆ2 for Linear Regression: 0.4711263309877963


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [137]:
model_dtr = train_test_model(X_train, X_test, y_train, y_test, "Decision Tree Regression")

MAE for Decision Tree Regression: 8.224714656322481
MSE for Decision Tree Regression: 113.3867405058904
Rˆ2 for Decision Tree Regression: 0.5443813577793883


In [139]:
model_rfr = train_test_model(X_train, X_test, y_train, y_test, "Random Forest Regression")

MAE for Random Forest Regression: 8.01864744484889
MSE for Random Forest Regression: 107.76830323831679
Rˆ2 for Random Forest Regression: 0.5669577608740746


In [138]:
model_dtr.best_estimator_

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=8,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [140]:
model_rfr.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [144]:
y_pred_train_dtr = model_dtr.best_estimator_.predict(X_train)

print(f"MAE for Decision Tree Regression on training set: {mean_absolute_error(y_train, y_pred_train_dtr)}")
print(f"MSE for Decision Tree Regression on training set: {mean_squared_error(y_train, y_pred_train_dtr)}")
print(f"Rˆ2 for Decision Tree Regression on training set: {r2_score(y_train, y_pred_train_dtr)}")

MAE for Decision Tree Regression: 7.635512596032477
MSE for Decision Tree Regression: 98.49768284343183
Rˆ2 for Decision Tree Regression: 0.5995698305865005


In [145]:
y_pred_train_rfr = model_rfr.best_estimator_.predict(X_train)

print(f"MAE for Random Forest Regression on training set: {mean_absolute_error(y_train, y_pred_train_rfr)}")
print(f"MSE for Random Forest Regression on training set: {mean_squared_error(y_train, y_pred_train_rfr)}")
print(f"Rˆ2 for Random Forest Regression on training set: {r2_score(y_train, y_pred_train_rfr)}")

MAE for Random Forest Regression on training set: 7.006710152085295
MSE for Random Forest Regression on training set: 81.52532795197362
Rˆ2 for Random Forest Regression on training set: 0.6685688440489359
